This notebook is used to calculate the differential abundance of OTUs in soils with PyOM or stover additions, as compared to soil only plots

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
# loading R packages
library(DESeq2)
library(phyloseq)
library(plyr); library(dplyr)
library(ggplot2)
library(foreach)
library(doParallel)
library(RColorBrewer)
library(gridExtra)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, union, unique,
    unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package:

In [9]:
%%R
#Creating the physeq object from our tree and biom table with taxonimic and metadata already included, telling it \
# that the taxonomy is from greengenes database, so it will recognize the samples
physeq = import_biom("../data/otu_table_tax_no211_meta.biom")
sample.data = import_qiime_sample_data("../data/MapFile.txt")
# combines the otu table and the mapping file
physeq = merge_phyloseq(physeq,sample.data)
sample.data = sample_data(physeq)

In [ ]:
%%R
physeq.pruned <- prune_samples((Day == Day)& ((Amdmt == Amdmt) | (Amdmt == 0)), physeq)

In [36]:
%%R
#run deseq on each subset (mdply)
#indep filtering
#adjust p values
# write to file in data
Dif_Abund = function(Day, Amdmt){
    physeq.pruned <- prune_samples((sample.data$Day == Day)& ((sample.data$Amdmt == Amdmt) | (sample.data$Amdmt == 0)), physeq)
    dseq = phyloseq_to_deseq2(physeq.pruned, ~Amdmt)
    dseq = DESeq(dseq, quiet = TRUE, fitType = "local")
    results = results(dseq)
    results$Day = Day
    results$Amdmt = Amdmt
    results = data.frame(results$Day,results$Amdmt,rownames(results),results$baseMean,results$log2FoldChange,results$lfcSE,results$pvalue)
    colnames(results) = c("Day","Amdmt","OTU","baseMean","l2FC","SE","pvalue")
    results
    }

In [40]:
%%R
variables <- matrix(c(1,1,1,2,2,1,2,2),nrow=4,ncol=2)
# Creates a matrix coding for all possible day and treatment combinations
# i.e., days 12 and 82, for PyOM and stover

In [ ]:
%%R
DA<-mdply(variables,Dif_Abund)

In [ ]:
%%R
head(DA)

In [ ]:
%%R
dim(DA)